In [ ]:
import xml.etree.ElementTree as ET 
import gzip
from xml.etree.cElementTree import ElementTree

In [ ]:
initialPlanFile = '/Users/MathildeMoliner/Documents/Polytechnique/MATSim/create_input/decompress/input_plans.xml.gz'
treePopulation = ET.parse(gzip.open(initialPlanFile,'r'))
population  = treePopulation.getroot()

In [ ]:
motorHousehold=[]
privateHousehold=[]
linkHousehold=[]
taxiUser=0

# Etablir la liste des ménages motorisés
for person in population.iter('person'):
    carAvail=person[0][2]
    householdId=person[0][5]
    if carAvail.text!='never':
        carAvail.text='never'
        if householdId.text not in motorHousehold: 
            motorHousehold.append(householdId.text)

In [ ]:
for person in population.iter('person'):
    householdId=person[0][5]
    incomeClass=person[0][6] 
    linkHome=person[1][0].get('link') 
    if ((incomeClass.text == '5') and (householdId.text in motorHousehold)): #condition pour posséder private_AV
        subpop=ET.SubElement(person[0],'attribute', attrib={'name': "subpopulation", 'class':"java.lang.String"})
        subpop.text="private"
        if householdId.text not in privateHousehold: #récuperer householdId et linkHome pour les véhicules
            privateHousehold.append(householdId.text)
            linkHousehold.append(linkHome)
        for leg in person[1].iter('leg'):
            if leg.get('mode') == 'car':
                leg.set('mode','private_AV')
                leg[0][0].text='private_AV'
                if leg[1].get('start_link') == leg[1].get('end_link'):
                    leg.set('mode','walk')
                    leg[0][0].text='walk'
                    leg[1].text=''
            if leg.get('mode') == 'car_passenger':
                leg.set('mode','private_AV')
                leg[0][0].text='private_AV'
                if leg[1].get('start_link') == leg[1].get('end_link'):
                    leg.set('mode','walk')
                    leg[0][0].text='walk'
                    leg[1].text=''
    else: #utilisateurs de robotaxis
        subpop=ET.SubElement(person[0],'attribute', attrib={'name': "subpopulation", 'class':"java.lang.String"})
        subpop.text="unmotorized"
        taxiUser+=1
        for leg in person[1].iter('leg'):
            if leg.get('mode') == 'car':
                leg.set('mode','drt')
                leg[0][0].text='drt'
                if leg[1].get('start_link') == leg[1].get('end_link'):
                    leg.set('mode','walk')
                    leg[0][0].text='walk'
                    leg[1].text=''
                else :
                    leg.set('mode','drt') #avec drt <route> est different de car (ce n'est pas la liste des liens mais les paramètres d'attente et detour autorisé)
                    leg.remove(leg[0])
                    leg.remove(leg[0])
            if leg.get('mode') == 'car_passenger':
                leg.set('mode','drt')
                leg[0][0].text='drt'
                if leg[1].get('start_link') == leg[1].get('end_link'):
                    leg.set('mode','walk')
                    leg[0][0].text='walk'
                    leg[1].text=''
                else :
                    leg.set('mode','drt')
                    leg.remove(leg[0])
                    leg.remove(leg[0])


In [ ]:
print("Nombre de PAV :", len(privateHousehold))
print("Nombre de DRT :", int(taxiUser/10))

In [ ]:

modifiedPlanFile = 'data/scenario1/population_sce1.xml.gz'
f = gzip.open(modifiedPlanFile, 'wb')
f.write('<?xml version="1.0" encoding="utf-8"?><!DOCTYPE population SYSTEM "http://www.matsim.org/files/dtd/population_v6.dtd">'.encode('utf8'))
ElementTree(population).write(f) 
f.close()

In [ ]:
# Utiliser CreateTaxiVehicles.java pour créer le fichier de taxis avec int(taxiUser/10)
#  en modifiant: int numberofVehicles = 12923;
#                String networkfile = "newNetwork.xml";
#		         String taxisFile = "/Users/MathildeMoliner/Documents/Polytechnique/MATSim/create_input/data/scenario2/taxis.xml";
#           (l72) "drt"


In [ ]:
# Utiliser CreateTaxiVehicles.java pour créer le fichier de véhicules privés selon len(motorHousehold)
#  en modifiant: int numberofVehicles = 59502;
#                String networkfile = "/Users/MathildeMoliner/Documents/Polytechnique/MATSim/create_input/data/input_files/montreal_network.xml.gz";
#		         String taxisFile = "/Users/MathildeMoliner/Documents/Polytechnique/MATSim/create_input/decompress/private_vehicles_sce3.xml";
		
treeVehicles = ET.parse('./decompress/private_vehicles_sce3.xml')
rootVehicles = treeVehicles.getroot()

In [ ]:
i=0
for taxi in rootVehicles.iter('vehicle'):
    taxi.set('id','taxi_' + str(motorHousehold[i]))
    taxi.set('start_link',linkHousehold[i])
    i+=1

In [ ]:
g = gzip.open('data/scenario3/private_vehicles_sce3.xml.gz', 'wb')
g.write('<?xml version="1.0" encoding="utf-8"?><!DOCTYPE vehicles SYSTEM "http://matsim.org/files/dtd/dvrp_vehicles_v1.dtd">'.encode('utf8'))
ElementTree(rootVehicles).write(g,'utf-8')
g.close()